In [7]:
import pandas as pd
df = pd.read_csv('D:/parenti_veri/definitivo_covid19.csv')

In [30]:
import re
def extract_aff(total_string):
    aff_list = []
    aff = re.findall("(?s)affil std {(.*?)}", total_string)
    for el in aff:
        aff_list.append(el)
    if len(aff)>0 :
        return aff_list
    else: 
        aff2 = re.findall("(?s)affil str \"(.*?)\"", total_string)
        for el in aff:
            aff_list.append(clean_long(el))
    if len(aff)==0 :
        aff_list.append(total_string)
    return aff_list

def extract_sub_country(total_string):
    aff_list = []
    aff = re.findall("(?s)subtype country,(.*?)}", total_string)
    for el in aff:
        aff_list.append(el)

    return aff_list


def clean_sub_country(country):
        clean = re.findall("(?s)name \"(.*?)\"", country)   
        if clean:
            splitter = clean[0].split(":")   
            return splitter[0]
        else:
            return "no_country"


def extract_collectedby(total_string):
    aff_list = []
    aff = re.findall("(?s)subtype collected-by,(.*?)}", total_string)
    for el in aff:
        aff_list.append(el)

    return aff_list


def extract_country(affiliations):
            countries = []
            for el in affiliations:
                country = re.findall("(?s)affil str \"(.*?)\"", el)
                for el in country:
                   countries.append(clean_long(el))


            if(len(countries)==0):
                for el in affiliations:
                    country = re.search("(?s)country \"(.*?)\"", el)
                    if(country != None):   
                     countries.append(extract_tra_virgolette(el[int(country.span()[0]):int(country.span()[1])]))
                    if(country == None):
                        countries.append("no country")            
            return countries

def extract_aff_precisa(affiliations):
            countries = []
            for el in affiliations:
                country = re.findall("(?s)affil \"(.*?)\"", el)
                for el in country:
                    countries.append(clean_long(el))
            if not countries:
                for el in affiliations:
                    country = re.findall("(?s)affil str \"(.*?)\"", el)
                    for elm in country:
                        countries.append(clean_long(elm))    
            return countries            


def remove_digits(row):
    countries = row['country']
    for el in countries:
        regex = re.findall("\d", el)
        if regex:
            countries.remove(el)
    return countries
 
     

def extract_first(countries):
    if countries:
        return countries[0]
    else:
        return "no_country" 

def extract_last(countries):
    if countries:
        return countries[-1]
    else:
        return "no_country"  
def countries_fl(riga):
    lista = []
    lista.append(riga['first_country'])
    lista.append(riga['last_country'])
    return lista                      

def clean_long(long_string):
    if len(long_string)>50:
        result = long_string.rsplit(None, 1)[-1]
        return result        
    else:
        return long_string

def extract_tra_virgolette(stringa):
     country = re.search("(?s)\"(.*?)\"", stringa)
     return stringa[int(country.span()[0]+1):int(country.span()[1]-1)]

def controllo_uguali(row):
    if row['first_country'] == row['last_country']:
        return "uguali"
    else:
        return "diversi"

def len_affiliations(row):
    return len(row['country'])


In [9]:
df['affiliations'] = df.apply(lambda row: extract_aff(row['total_string']),axis=1)
df['affiliations']

0         [\n              affil "US CDC",\n            ...
1         [\n              affil "US CDC",\n            ...
2         [\n              affil "US CDC",\n            ...
3         [\n              affil "US CDC",\n            ...
4         [\n              affil "US CDC",\n            ...
                                ...                        
321932    [\n              affil "Texas Department of St...
321933    [\n              affil "Texas Department of St...
321934    [\n              affil "Texas Department of St...
321935    [\n              affil "Texas Department of St...
321936    [\n              affil "Texas Department of St...
Name: affiliations, Length: 321937, dtype: object

In [31]:
df['submission_country'] = df.apply(lambda row: extract_sub_country(row['total_string']),axis=1)
df['submission_country']

0             [\n          name "USA"\n        ]
1             [\n          name "USA"\n        ]
2             [\n          name "USA"\n        ]
3             [\n          name "USA"\n        ]
4             [\n          name "USA"\n        ]
                           ...                  
321932    [\n          name "USA: TX"\n        ]
321933    [\n          name "USA: TX"\n        ]
321934    [\n          name "USA: TX"\n        ]
321935    [\n          name "USA: TX"\n        ]
321936    [\n          name "USA: TX"\n        ]
Name: submission_country, Length: 321937, dtype: object

In [34]:

B = df['submission_country_clean'].value_counts()
B.to_csv("subcountryclean2.csv",mode='a')

In [32]:
df['submission_country_clean'] = df.apply(lambda row: clean_sub_country(extract_first(row['submission_country'])),axis=1)
df['submission_country_clean']

0         USA
1         USA
2         USA
3         USA
4         USA
         ... 
321932    USA
321933    USA
321934    USA
321935    USA
321936    USA
Name: submission_country_clean, Length: 321937, dtype: object

In [16]:

df['first_subtype'] = df.apply(lambda row: extract_first(row['subtype']),axis=1)
B = df['first_subtype'].value_counts()
B.to_csv("subtype.csv",mode='a')

In [23]:
df['collectedby'] = df.apply(lambda row: extract_collectedby(row['total_string']),axis=1)
df['first_collectedby'] = df.apply(lambda row: extract_first(row['collectedby']),axis=1)


In [24]:
B = df['first_collectedby'].value_counts()
B.to_csv("subtypecollectedby.csv",mode='a')

In [6]:
df['affiliations_clean'] = df.apply(lambda row: extract_aff_precisa(row['affiliations']),axis=1)
df['affiliations_clean']

0         [US CDC]
1         [US CDC]
2         [US CDC]
3         [US CDC]
4         [US CDC]
            ...   
321932     [group]
321933     [group]
321934     [group]
321935     [group]
321936     [group]
Name: affiliations_clean, Length: 321937, dtype: object

In [7]:
df['first_aff'] = df.apply(lambda row: extract_first(row['affiliations_clean']),axis=1)
df['first_aff'].value_counts()

Kingdom                                       123309
Centers for Disease Control and Prevention    108730
group                                          13699
Immunity                                        7728
Group                                           6512
                                               ...  
RESEARCH                                           1
Bangladesh                                         1
Nigerian Institute of Medical Research             1
UMR190                                             1
Institute of Meat Hygiene and Technology           1
Name: first_aff, Length: 266, dtype: int64

In [15]:
df['country'] = df.apply(lambda row: extract_country(row['affiliations']),axis=1)
df['country_clean'] = df.apply(lambda row: remove_digits(row),axis=1)

df['country_clean']

0         [USA]
1         [USA]
2         [USA]
3         [USA]
4         [USA]
          ...  
321932    [USA]
321933    [USA]
321934    [USA]
321935    [USA]
321936    [USA]
Name: country_clean, Length: 321937, dtype: object

In [16]:
df['first_country'] = df.apply(lambda row: extract_first(row['country']),axis=1)
df['last_country'] = df.apply(lambda row: extract_last(row['country']),axis=1)
df['Countries_FL'] = df.apply(lambda row: countries_fl(row),axis=1)


In [17]:
df['controllo'] = df.apply(lambda row: controllo_uguali(row),axis=1)
df['len'] = df.apply(lambda row: len_affiliations(row),axis=1)

In [18]:
df['controllo'].value_counts()

uguali     321925
diversi        12
Name: controllo, dtype: int64

In [9]:
df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,affiliations,country,country_clean,first_country,last_country,Countries_FL,controllo,len
0,0,0,0.0,0.0,"[\n affil ""US CDC"",\n ...",[USA],[USA],USA,USA,"[USA, USA]",uguali,1
1,1,1,1.0,1.0,"[\n affil ""US CDC"",\n ...",[USA],[USA],USA,USA,"[USA, USA]",uguali,1
2,2,2,2.0,2.0,"[\n affil ""US CDC"",\n ...",[USA],[USA],USA,USA,"[USA, USA]",uguali,1
3,3,3,3.0,3.0,"[\n affil ""US CDC"",\n ...",[USA],[USA],USA,USA,"[USA, USA]",uguali,1
4,4,4,4.0,4.0,"[\n affil ""US CDC"",\n ...",[USA],[USA],USA,USA,"[USA, USA]",uguali,1
...,...,...,...,...,...,...,...,...,...,...,...,...
321932,994,994,NaN,NaN,"[\n affil ""Texas Department of St...",[USA],[USA],USA,USA,"[USA, USA]",uguali,1
321933,995,995,NaN,NaN,"[\n affil ""Texas Department of St...",[USA],[USA],USA,USA,"[USA, USA]",uguali,1
321934,996,996,NaN,NaN,"[\n affil ""Texas Department of St...",[USA],[USA],USA,USA,"[USA, USA]",uguali,1
321935,997,997,NaN,NaN,"[\n affil ""Texas Department of St...",[USA],[USA],USA,USA,"[USA, USA]",uguali,1


In [ ]:
df['affiliations'].value_counts()

In [21]:
A = df['first_country'].value_counts()
A.to_csv("cov19firstcou.csv",mode='a')

In [35]:
import pycountry
import time
cl=list(pycountry.countries)
countries_list=[]
count=-1
for paese in cl:
    count+=1
    try:
        name=list(pycountry.countries)[count].name
        countries_list.append(name)
    except AttributeError:
        time.sleep(1)
    try:
        common_name=list(pycountry.countries)[count].common_name
        countries_list.append(common_name)
    except AttributeError:
        time.sleep(1)
    try:
        official_name=list(pycountry.countries)[count].official_name
        countries_list.append(official_name)
    except AttributeError:
        time.sleep(1)
    try:
        alpha_2=list(pycountry.countries)[count].alpha_2
        countries_list.append(alpha_2)
    except AttributeError:
        time.sleep(1)
    try:
        alpha_3=list(pycountry.countries)[count].alpha_3
        countries_list.append(alpha_3)
    except AttributeError:
        time.sleep(1)

len(countries_list) #in questa lista ci sono dentro tutti i riferimenti possibili ai paesi

926

In [99]:
countries_list.append('UK') #manca UK, lo aggiungiamo
countries_list.append('PR China')
countries_list.append('England')
countries_list.append('Perù')
countries_list.append('México')
countries_list.append('España')
countries_list.append('United States of America')
countries_list.append('Brasil')
countries_list.append('Deutschland') 
countries_list.append('Iran')
countries_list.append('Russia')
countries_list.append('Republic of Korea')
countries_list.append('Italia')
countries_list.append('Korea')
countries_list.append('UNITED STATES')
countries_list.append('SPAIN')
countries_list.append('GERMANY')
countries_list.append('belgique')

In [100]:
ustates = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY",'New York (JBB)','New York (PD)',"Alaska", "Alabama", "Arkansas", "American Samoa", "Arizona", "California", "Colorado", "Connecticut", "District ", "of Columbia", "Delaware", "Florida", "Georgia", "Guam", "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", "Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", "New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Puerto Rico", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", "Virgin Islands", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]

city=pd.read_csv("https://raw.githubusercontent.com/grammakov/USA-cities-and-states/master/us_cities_states_counties.csv",delimiter='|')
uscity=city['City'].values.tolist()
uscity=list(dict.fromkeys(uscity))
len(uscity)

ustates=ustates+uscity
len(ustates)


cities=pd.read_csv("/parenti_veri/world-cities_csv.csv",converters={'Affiliations': eval})
print(cities.head())

chinalist = cities[cities['country']=='China']['name'].tolist()
fralist = cities[cities['country']=='France']['name'].tolist()
spagnalist = cities[cities['country']=='Spain']['name'].tolist()
gerlist = cities[cities['country']=='Germany']['name'].tolist()
nedlist = cities[cities['country']=='Netherlands']['name'].tolist()
japlist = cities[cities['country']=='Japan']['name'].tolist()
italist = cities[cities['country']=='Italy']['name'].tolist()
uklist = cities[cities['country']=='United Kingdom']['name'].tolist()

               name               country          subcountry  geonameid
0      les Escaldes               Andorra  Escaldes-Engordany    3040051
1  Andorra la Vella               Andorra    Andorra la Vella    3041563
2    Umm al Qaywayn  United Arab Emirates      Umm al Qaywayn     290594
3    Ras al-Khaimah  United Arab Emirates     Raʼs al Khaymah     291074
4      Khawr Fakkān  United Arab Emirates        Ash Shāriqah     291696


In [101]:
p1 = df

In [125]:
#PROVIAMO A TENERE SEPRATI I VALORI DI INVESTIMENTO E AUTORE PRINCIPALE

diz_cov_country={}
id_nonaz=[]

for index,row in p1.iterrows():
    inout=0 #per scovare gli indici che non hanno paese ma solo laboratorio
    count=-1
    for el in row['Countries_FL']:
        count+=1
        if row['Countries_FL'][count] in countries_list:
            inout+=1
            if row['Countries_FL'][count] in diz_cov_country:
                diz_cov_country[row['Countries_FL'][count]][count]+=1
            else:
                diz_cov_country[row['Countries_FL'][count]]=[0,0]
                diz_cov_country[row['Countries_FL'][count]][count]+=1
        elif row['Countries_FL'][count] in ustates:
            inout+=1
            if 'USA' in diz_cov_country:
                diz_cov_country['USA'][count]+=1
            else:
                diz_cov_country['USA']=[0,0]
                diz_cov_country['USA'][count]+=1
                
        elif row['Countries_FL'][count] in chinalist:
                    inout+=1
                    if 'China' in diz_cov_country:
                        diz_cov_country['China'][count]+=1
                    else:
                        diz_cov_country['China']=[0,0]
                        diz_cov_country['China'][count]+=1
                
        elif row['Countries_FL'][count] in fralist:
                    inout+=1
                    if 'France' in diz_cov_country:
                        diz_cov_country['France'][count]+=1
                    else:
                        diz_cov_country['France']=[0,0]
                        diz_cov_country['France'][count]+=1
                
        elif row['Countries_FL'][count] in spagnalist:
                    inout+=1
                    if 'Spain' in diz_cov_country:
                        diz_cov_country['Spain'][count]+=1
                    else:
                        diz_cov_country['Spain']=[0,0]
                        diz_cov_country['Spain'][count]+=1
                    
        elif row['Countries_FL'][count] in gerlist:
                    inout+=1
                    if 'Germany' in diz_cov_country:
                        diz_cov_country['Germany'][count]+=1
                    else:
                        diz_cov_country['Germany']=[0,0]
                        diz_cov_country['Germany'][count]+=1
                        
        elif row['Countries_FL'][count] in nedlist:
                    inout+=1
                    if 'Netherlands' in diz_cov_country:
                        diz_cov_country['Netherlands'][count]+=1
                    else:
                        diz_cov_country['Netherlands']=[0,0]
                        diz_cov_country['Netherlands'][count]+=1
                        
        elif row['Countries_FL'][count] in japlist:
                    inout+=1
                    if 'Japan' in diz_cov_country:
                        diz_cov_country['Japan'][count]+=1
                    else:
                        diz_cov_country['Japan']=[0,0]
                        diz_cov_country['Japan'][count]+=1
                        
                        
        elif row['Countries_FL'][count] in italist:
                    inout+=1
                    if 'Italy' in diz_cov_country:
                        diz_cov_country['Italy'][count]+=1
                    else:
                        diz_cov_country['Italy']=[0,0]
                        diz_cov_country['Italy'][count]+=1
                        
                        
        elif row['Countries_FL'][count] in uklist:
                    inout+=1
                    if 'United Kingdom' in diz_cov_country:
                        diz_cov_country['United Kingdom'][count]+=1
                    else:
                        diz_cov_country['United Kingdom']=[0,0]
                        diz_cov_country['United Kingdom'][count]+=1
                
                

                
                
                
        else:
            spl= row['Countries_FL'][count].split(" ")
            for elemento in spl:
                if elemento in countries_list:
                    inout+=1
                    if elemento in diz_cov_country:
                        diz_cov_country[elemento][count]+=1
                    else:
                        diz_cov_country[elemento]=[0,0]
                        diz_cov_country[elemento][count]+=1
                        
                elif elemento in ustates:
                    inout+=1
                    if 'USA' in diz_cov_country:
                        diz_cov_country['USA'][count]+=1
                    else:
                        diz_cov_country['USA']=[0,0]
                        diz_cov_country['USA'][count]+=1
                        
                elif elemento in chinalist:
                    inout+=1
                    if 'China' in diz_cov_country:
                        diz_cov_country['China'][count]+=1
                    else:
                        diz_cov_country['China']=[0,0]
                        diz_cov_country['China'][count]+=1
                
                elif elemento in fralist:
                    inout+=1
                    if 'France' in diz_cov_country:
                        diz_cov_country['France'][count]+=1
                    else:
                        diz_cov_country['France']=[0,0]
                        diz_cov_country['France'][count]+=1
                
                elif elemento in spagnalist:
                    inout+=1
                    if 'Spain' in diz_cov_country:
                        diz_cov_country['Spain'][count]+=1
                    else:
                        diz_cov_country['Spain']=[0,0]
                        diz_cov_country['Spain'][count]+=1
                    
                elif elemento in gerlist:
                    inout+=1
                    if 'Germany' in diz_cov_country:
                        diz_cov_country['Germany'][count]+=1
                    else:
                        diz_cov_country['Germany']=[0,0]
                        diz_cov_country['Germany'][count]+=1
                        
                elif elemento in nedlist:
                    inout+=1
                    if 'Netherlands' in diz_cov_country:
                        diz_cov_country['Netherlands'][count]+=1
                    else:
                        diz_cov_country['Netherlands']=[0,0]
                        diz_cov_country['Netherlands'][count]+=1
                        
                elif elemento in japlist:
                    inout+=1
                    if 'Japan' in diz_cov_country:
                        diz_cov_country['Japan'][count]+=1
                    else:
                        diz_cov_country['Japan']=[0,0]
                        diz_cov_country['Japan'][count]+=1
                        
                        
                elif elemento in italist:
                    inout+=1
                    if 'Italy' in diz_cov_country:
                        diz_cov_country['Italy'][count]+=1
                    else:
                        diz_cov_country['Italy']=[0,0]
                        diz_cov_country['Italy'][count]+=1
                        
                        
                elif elemento in uklist:
                    inout+=1
                    if 'United Kingdom' in diz_cov_country:
                        diz_cov_country['United Kingdom'][count]+=1
                    else:
                        diz_cov_country['United Kingdom']=[0,0]
                        diz_cov_country['United Kingdom'][count]+=1
                
                

                
                
                else:
                    try:
                        dop=str(spl[0]+' '+spl[1])
                        if dop in countries_list:
                            inout+=1
                            if dop in diz_cov_country:
                                diz_cov_country[dop][count]+=1
                            else:
                                diz_cov_country[dop]=[0,0]
                                diz_cov_country[dop][count]+=1
                            
                    except IndexError:
                        inout+=0
                        
    if inout < 2:
        id_nonaz.append(index)

#round dei valori float per eccesso:
#for key in diz_cov_country:
#    diz_cov_country[key]=int(np.ceil(diz_cov_country[key]))   

#ordinamento del dizionario   
sort_diz=sorted(diz_cov_country.items(),key=lambda item: item[1], reverse=True)
sort_diz      

[('China', [9579, 9584]),
 ('USA', [6367, 6379]),
 ('Brazil', [1312, 1305]),
 ('Italy', [1280, 1275]),
 ('Thailand', [1058, 1058]),
 ('Korea', [788, 792]),
 ('NL', [620, 373]),
 ('US', [618, 386]),
 ('Germany', [586, 589]),
 ('Netherlands', [569, 570]),
 ('Hong Kong', [538, 541]),
 ('KR', [490, 490]),
 ('Japan', [419, 806]),
 ('Taiwan', [419, 418]),
 ('France', [414, 475]),
 ('Malaysia', [389, 389]),
 ('Egypt', [365, 372]),
 ('Sweden', [362, 365]),
 ('Canada', [347, 355]),
 ("People's Republic of China", [342, 342]),
 ('Australia', [287, 301]),
 ('UK', [278, 281]),
 ('Republic of Korea', [266, 266]),
 ('Iran', [258, 253]),
 ('India', [252, 249]),
 ('South Africa', [194, 194]),
 ('Luxembourg', [183, 226]),
 ('Slovenia', [158, 158]),
 ('Viet Nam', [147, 147]),
 ('Kingdom of Saudi Arabia', [145, 145]),
 ('Turkey', [141, 139]),
 ('FR', [138, 138]),
 ('Russia', [127, 116]),
 ('Singapore', [123, 121]),
 ('AND', [118, 118]),
 ('Saudi Arabia', [110, 108]),
 ('Uruguay', [106, 106]),
 ('Poland',

In [126]:
count=0
diz_aus={}
for key in diz_cov_country:
    
    if len(key) == 2:
        try:
            country = pycountry.countries.get(alpha_2=key)
            name=country.name
            if name in diz_aus:
                diz_aus[name]+=diz_cov_country[key]
            if name not in diz_aus:
                diz_aus[name]=diz_cov_country[key]

        except AttributeError:
            count+=1
            
    if len(key) == 3:
        try:
            country = pycountry.countries.get(alpha_3=key)
            name=country.name
            if name in diz_aus:
                diz_aus[name]+=diz_cov_country[key]
            if name not in diz_aus:
                diz_aus[name]=diz_cov_country[key]
                
        except AttributeError:
            count+=1

 
diz_aus 

{'United States': [6367, 6379, 618, 386],
 'Japan': [76, 76],
 'Netherlands': [620, 373],
 'Andorra': [118, 118],
 'Singapore': [33, 33],
 'China': [39, 39],
 'Morocco': [0, 7],
 'United Kingdom': [77, 36],
 'France': [138, 138],
 'Canada': [47, 0],
 'Cyprus': [94, 0],
 'Cuba': [14, 4],
 'Taiwan, Province of China': [16, 0],
 'Colombia': [13, 13],
 "Lao People's Democratic Republic": [54, 54],
 'Bouvet Island': [13, 13],
 'Italy': [7, 0],
 'Switzerland': [9, 0],
 'Germany': [1, 1],
 'Indonesia': [26, 26],
 'Saudi Arabia': [24, 24],
 'Spain': [21, 21],
 'Korea, Republic of': [490, 490]}

In [127]:
for key in diz_aus:
    count=-1
    pari=0
    disp=0
    if len(diz_aus[key]) > 2:
        for el in diz_aus[key]:
            count+=1
            if count==0 or count==2:
                pari+=diz_aus[key][count]
            if count==1 or count==3:
                disp+=diz_aus[key][count]
        diz_aus[key]=[pari,disp]
diz_aus

{'United States': [6985, 6765],
 'Japan': [76, 76],
 'Netherlands': [620, 373],
 'Andorra': [118, 118],
 'Singapore': [33, 33],
 'China': [39, 39],
 'Morocco': [0, 7],
 'United Kingdom': [77, 36],
 'France': [138, 138],
 'Canada': [47, 0],
 'Cyprus': [94, 0],
 'Cuba': [14, 4],
 'Taiwan, Province of China': [16, 0],
 'Colombia': [13, 13],
 "Lao People's Democratic Republic": [54, 54],
 'Bouvet Island': [13, 13],
 'Italy': [7, 0],
 'Switzerland': [9, 0],
 'Germany': [1, 1],
 'Indonesia': [26, 26],
 'Saudi Arabia': [24, 24],
 'Spain': [21, 21],
 'Korea, Republic of': [490, 490]}

In [128]:
dup=list(diz_cov_country.keys())
import collections
print([item for item, count in collections.Counter(dup).items() if count > 1])

[]


In [129]:
for key in diz_aus:
    for chiave in diz_cov_country:
        if key == chiave:
            diz_cov_country[chiave]+=diz_aus[key]
sort_diz_fin=sorted(diz_cov_country.items(),key=lambda item: item[0], reverse=True)

for key in diz_cov_country:
    count=-1
    pari=0
    disp=0
    if len(diz_cov_country[key]) > 2:
        for el in diz_cov_country[key]:
            count+=1
            if count==0 or count==2:
                pari+=diz_cov_country[key][count]
            if count==1 or count==3:
                disp+=diz_cov_country[key][count]
        diz_cov_country[key]=[pari,disp]
diz_cov_country

{'Brasil': [92, 84],
 'China': [9618, 9623],
 'Canada': [394, 355],
 'Egypt': [365, 372],
 'India': [252, 249],
 'France': [552, 613],
 'Korea': [788, 792],
 'USA': [6985, 6765],
 'United Kingdom': [150, 108],
 'Gabon': [21, 21],
 'Uruguay': [106, 106],
 'Thailand': [1058, 1058],
 'Republic of Korea': [266, 266],
 'Iran': [258, 253],
 'New Zealand': [10, 10],
 'Turkey': [141, 139],
 'Chile': [40, 40],
 'Czech Republic': [10, 10],
 'Germany': [587, 590],
 "People's Republic of China": [342, 342],
 'Malaysia': [389, 389],
 'United States of America': [1, 1],
 'Italy': [1287, 1275],
 'Taiwan': [419, 418],
 'Brazil': [1312, 1305],
 'UK': [278, 281],
 'US': [618, 386],
 'Luxembourg': [183, 226],
 'Slovenia': [158, 158],
 'Spain': [80, 77],
 'Argentina': [29, 29],
 'Israel': [24, 28],
 'Netherlands': [1189, 943],
 'JP': [76, 76],
 'NL': [620, 373],
 'AND': [118, 118],
 'SG': [33, 33],
 'CN': [39, 39],
 'Greece': [72, 69],
 'Sweden': [362, 365],
 'Russia': [127, 116],
 'Japan': [495, 882],
 '

In [130]:
sorted(diz_cov_country.items(),key=lambda item: item[1], reverse=True)

[('China', [9618, 9623]),
 ('United States', [6986, 6765]),
 ('USA', [6985, 6765]),
 ('Brazil', [1312, 1305]),
 ('Italy', [1287, 1275]),
 ('Netherlands', [1189, 943]),
 ('Thailand', [1058, 1058]),
 ('Korea', [788, 792]),
 ('NL', [620, 373]),
 ('US', [618, 386]),
 ('Germany', [587, 590]),
 ('France', [552, 613]),
 ('Hong Kong', [538, 541]),
 ('Japan', [495, 882]),
 ('Korea, Republic of', [493, 493]),
 ('KR', [490, 490]),
 ('Taiwan', [419, 418]),
 ('Canada', [394, 355]),
 ('Malaysia', [389, 389]),
 ('Egypt', [365, 372]),
 ('Sweden', [362, 365]),
 ("People's Republic of China", [342, 342]),
 ('Australia', [287, 301]),
 ('UK', [278, 281]),
 ('Republic of Korea', [266, 266]),
 ('Iran', [258, 253]),
 ('India', [252, 249]),
 ('South Africa', [194, 194]),
 ('Luxembourg', [183, 226]),
 ('Slovenia', [158, 158]),
 ('Singapore', [156, 154]),
 ('United Kingdom', [150, 108]),
 ('Viet Nam', [147, 147]),
 ('Kingdom of Saudi Arabia', [145, 145]),
 ('Turkey', [141, 139]),
 ('FR', [138, 138]),
 ('Saudi A

In [131]:
diz_cov_country['UnitedStates2']=diz_cov_country['United States']+diz_cov_country['United States of America']+diz_cov_country['USA']+diz_cov_country['US']
diz_cov_country['China2']=diz_cov_country['China'] + diz_cov_country['PR China'] + diz_cov_country['Hong Kong'] + diz_cov_country["People's Republic of China"] + diz_cov_country['Taiwan']
diz_cov_country['Italy2']=diz_cov_country['Italy'] + diz_cov_country['Italia']
diz_cov_country['Netherlands2']=diz_cov_country['Netherlands'] + diz_cov_country['NL'] 
diz_cov_country['France2']=diz_cov_country['France'] + diz_cov_country['FR'] 
diz_cov_country['UnitedKingdom2']=diz_cov_country['UK'] + diz_cov_country['United Kingdom'] + diz_cov_country['England']
diz_cov_country['Brazil2']=diz_cov_country['Brazil'] + diz_cov_country['Brasil']
diz_cov_country['Iran2']=diz_cov_country['Islamic Republic of Iran']  + diz_cov_country['Iran']
diz_cov_country['SaudiArabia2']=diz_cov_country['Saudi Arabia'] + diz_cov_country['Kingdom of Saudi Arabia']
diz_cov_country['Korea2']=diz_cov_country['Republic of Korea'] + diz_cov_country['Korea']

In [132]:
USA=['United States','United States of America','UNITED STATES','USA','US']
CHINA=['China','PR China','Hong Kong',"People's Republic of China",'Taiwan']
ITALIA=['Italy','Italia']
NL=['Netherlands','NL']
FRANCE=['France','FR']
UK=['United Kingdom of Great Britain and Northern Ireland','United Kingdom','England', 'UK']
GERMANIA=['Germany','GERMANY','Deutschland']
BRASILE=['Brazil','Brasil']
IRAN=['Islamic Republic of Iran','Iran']
ARABIA_SAUDITA=['Saudi Arabia','Kingdom of Saudi Arabia']
KOREA=['Republic of Korea','Korea']

#eliminazione chiavi vecchie non utili
for key, value in list(diz_cov_country.items()):
    if key in USA or key in CHINA or key in ITALIA or key in UK or key in GERMANIA or key in BRASILE or key in IRAN or key in ARABIA_SAUDITA or key in KOREA or key in NL or key in FRANCE: #poi gestisci manualmente UK
        del(diz_cov_country[key])
len(diz_cov_country)

91

In [133]:
diz_cov_country['United States'] = diz_cov_country.pop('UnitedStates2')
diz_cov_country['China'] = diz_cov_country.pop('China2')
diz_cov_country['Italy'] = diz_cov_country.pop('Italy2')
diz_cov_country['United Kingdom'] = diz_cov_country.pop('UnitedKingdom2')
diz_cov_country['Brazil'] = diz_cov_country.pop('Brazil2')
diz_cov_country['Iran'] = diz_cov_country.pop('Iran2')
diz_cov_country['Saudi Arabia'] = diz_cov_country.pop('SaudiArabia2')
diz_cov_country['Korea'] = diz_cov_country.pop('Korea2')
diz_cov_country['Netherlands'] = diz_cov_country.pop('Netherlands2')
diz_cov_country['France'] = diz_cov_country.pop('France2')

In [136]:
sorted_diz_fin=sorted(diz_cov_country.items(),key=lambda item: item[1], reverse=True)
sorted_diz_fin

[('China', [9618, 9623, 1, 1, 538, 541, 342, 342, 419, 418]),
 ('United States', [6986, 6765, 1, 1, 6985, 6765, 618, 386]),
 ('Brazil', [1312, 1305, 92, 84]),
 ('Italy', [1287, 1275, 4, 4]),
 ('Netherlands', [1189, 943, 620, 373]),
 ('Thailand', [1058, 1058]),
 ('France', [552, 613, 138, 138]),
 ('Japan', [495, 882]),
 ('Korea, Republic of', [493, 493]),
 ('KR', [490, 490]),
 ('Canada', [394, 355]),
 ('Malaysia', [389, 389]),
 ('Egypt', [365, 372]),
 ('Sweden', [362, 365]),
 ('Australia', [287, 301]),
 ('United Kingdom', [278, 281, 150, 108, 3, 3]),
 ('Korea', [266, 266, 788, 792]),
 ('India', [252, 249]),
 ('South Africa', [194, 194]),
 ('Luxembourg', [183, 226]),
 ('Slovenia', [158, 158]),
 ('Singapore', [156, 154]),
 ('Viet Nam', [147, 147]),
 ('Turkey', [141, 139]),
 ('Saudi Arabia', [134, 132, 145, 145]),
 ('Russia', [127, 116]),
 ('AND', [118, 118]),
 ('Uruguay', [106, 106]),
 ('Poland', [97, 88]),
 ('CY', [94, 0]),
 ('Spain', [80, 77]),
 ('Finland', [79, 79]),
 ('GB', [77, 36]),

In [137]:
#ultimo filtro
#sistemazione dizionario ausiliario
for key in diz_cov_country:
    count=-1
    pari=0
    disp=0
    if len(diz_cov_country[key]) > 2:
        for el in diz_cov_country[key]:
            count+=1
            if count==0 or count==2 or count==4 or count==6 or count==8:
                pari+=diz_cov_country[key][count]
            if count==1 or count==3 or count==5 or count==7 or count==9:
                disp+=diz_cov_country[key][count]
        diz_cov_country[key]=[pari,disp]
diz_cov_country

{'Canada': [394, 355],
 'Egypt': [365, 372],
 'India': [252, 249],
 'Gabon': [21, 21],
 'Uruguay': [106, 106],
 'Thailand': [1058, 1058],
 'New Zealand': [10, 10],
 'Turkey': [141, 139],
 'Chile': [40, 40],
 'Czech Republic': [10, 10],
 'Malaysia': [389, 389],
 'Luxembourg': [183, 226],
 'Slovenia': [158, 158],
 'Spain': [80, 77],
 'Argentina': [29, 29],
 'Israel': [24, 28],
 'JP': [76, 76],
 'AND': [118, 118],
 'SG': [33, 33],
 'CN': [39, 39],
 'Greece': [72, 69],
 'Sweden': [362, 365],
 'Russia': [127, 116],
 'Japan': [495, 882],
 'MA': [0, 7],
 'Ireland': [16, 16],
 'GB': [77, 36],
 'Tunisia': [8, 8],
 'CA': [47, 0],
 'CY': [94, 0],
 'Australia': [287, 301],
 'CUB': [14, 4],
 'Cuba': [15, 14],
 'TW': [16, 0],
 'CO': [13, 13],
 'Singapore': [156, 154],
 'Korea, Republic of': [493, 493],
 'Switzerland': [19, 23],
 'Poland': [97, 88],
 'LA': [54, 54],
 'Hungary': [64, 63],
 'South Africa': [194, 194],
 'Costa Rica': [4, 4],
 'Indonesia': [38, 38],
 'Denmark': [10, 6],
 'Mexico': [55, 5

In [138]:
listone=list(diz_cov_country.values())
chiavi=list(diz_cov_country.keys())
pa=[]
fun=[]
som=[]
listone
for li in listone:
    somma=0
    count=-1
    for el in li:
        count+=1
        if count==0:
            pa.append(el)
            somma+=el
        if count==1:
            fun.append(el)
            somma+=el
    som.append(somma)


countrycov_pub=pd.DataFrame(list(zip(chiavi,pa,fun,som)),columns=['Country','Principal_Author','Funding_country','Sum'])
countrycov_pub=countrycov_pub.sort_values(by=['Sum'],ascending=False).reset_index() 
countrycov_pub=countrycov_pub.drop('index',axis=1)
countrycov_pub=countrycov_pub[0:20]


#taglia india, brasile, iran, korea
#countrycov_pub=countrycov_pub.drop([4,8,10,14,17],axis=0).reset_index()
#countrycov_pub=countrycov_pub.drop('index',axis=1)
countrycov_pub

,Country,Principal_Author,Funding_country,Sum
0,United States,14590,13917,28507
1,China,10918,10925,21843
2,Netherlands,1809,1316,3125
3,Brazil,1404,1389,2793
4,Italy,1291,1279,2570
5,Thailand,1058,1058,2116
6,Korea,1054,1058,2112
7,France,690,751,1441
8,Japan,495,882,1377
9,"Korea, Republic of",493,493,986


In [30]:
test = df
test = test.drop(['total_string'], axis=1)
test = test.drop(['affiliations'], axis=1)

test.to_csv("testfirstlat.csv")

In [25]:
######################################################################################################

In [35]:

def extract_date(total_string):
    date = re.findall("(?s)collection-date(.*?)}", total_string) 
    return extract_year(date)
def fill_date(row):
    if not row['date']:
        date = re.findall("(?s)date(.*?)}", row['total_string']) 
        row['date'] = date
    else:
        row['date'] = row['date']

    return row['date']              
def extract_year(dates):
    years =[]
    for date in dates:
        year = re.findall("(?s)name \"(.*?)\"", date)
        years.append(year)
    return years
def full_date(date):
    lista = []
    for el in date:
        elstr = str(el)
        data = elstr[2:-2]
        if len(data) > 4:
            lista.append(el)
        else:
            lista.append("soloanno")
    return lista

def first_date(date):
    if date:
        return str(date[0])
    else:
        return "no_date"

def check_wrongformat(first_date):
     
     fd = first_date[2:-2]
     x = re.search('([0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9])',fd)
     if x:
        return first_date[2:-2]
     else:
         return "wrong format"

In [36]:

######data

import re

df['date'] = df.apply(lambda row: extract_date(row['total_string']),axis=1)
df['full_date'] = df.apply(lambda row: full_date(row['date']),axis=1)
df['first_date'] = df.apply(lambda row: first_date(row['full_date']),axis=1)

#df['first_date'].value_counts()

In [37]:
clean_df = df


In [38]:
clean_df['checked'] = clean_df.apply(lambda row: check_wrongformat(row['first_date']),axis=1)
#clean_df['checked'].value_counts()

In [45]:

clean_df = clean_df.drop(['date'], axis=1)

clean_df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,submission_country_clean,checked
0,0,0,0.0,0.0,USA,2021-04-14
1,1,1,1.0,1.0,USA,2021-04-12
2,2,2,2.0,2.0,USA,2021-04-05
3,3,3,3.0,3.0,USA,2021-04-08
4,4,4,4.0,4.0,USA,2021-04-06
...,...,...,...,...,...,...
321932,994,994,NaN,NaN,USA,2020-04-02
321933,995,995,NaN,NaN,USA,2020-03-31
321934,996,996,NaN,NaN,USA,2020-04-06
321935,997,997,NaN,NaN,USA,2020-04-07


In [47]:
!pip install openpyxl
import openpyxl
no_na.to_excel("covidpaesiedate.xlsx",sheet_name='foglio1')

In [30]:
no_na.to_csv("datepulitecovid.csv")